In [25]:
import pandas as pd

from definitions import ROOT_DIR
import pdpipe as pdp
import numpy as np

In [2]:
ROOT_DIR
PATH_ENERGY_IM_EXPORTS = ROOT_DIR / "data/EinAusFuhrElektrizität.csv"

In [37]:
df_raw = pd.read_csv(PATH_ENERGY_IM_EXPORTS, sep=";", decimal=",", header=6, encoding="utf-8")

In [39]:
pdp.df.replace(".", np.nan).apply(df_raw)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,MWh,MWh.1,MWh.2
0,2023,Januar,Frankreich,609151,801578,-192428
1,2023,Januar,Belgien und Luxemburg,104866,738715,-633849
2,2023,Januar,Niederlande,609089,1099654,-490565
3,2023,Januar,D�nemark,NaN,NaN,NaN
4,2023,Januar,Schweiz,NaN,NaN,NaN
...,...,...,...,...,...,...
119,2023,Dezember,Insgesamt,...,...,...
120,__________,NaN,NaN,NaN,NaN,NaN
121,In der Tabelle werden die physikalisch eingesp...,NaN,NaN,NaN,NaN,NaN
122,"� Statistisches Bundesamt (Destatis), 2023",NaN,NaN,NaN,NaN,NaN


In [54]:
pipeline = pdp.PdPipeline(
    [
        pdp.df.replace(to_replace=".", value=np.nan),
        pdp.df.replace(to_replace="...", value=np.nan),
        pdp.df.dropna(axis=0, how="any"),
        pdp.ColRename(
            rename_mapper={
                "Unnamed: 0": "year",
                "Unnamed: 1": "month",
                "Unnamed: 2": "country",
                "MWh": "import",
                "MWh.1": "export",
                "MWh.2": "net_import",
            }
        ),
        pdp.ColumnDtypeEnforcer(
            {
                "year": np.int16,
                "month": str,
                "country": str,
                "import": np.float32,
                "export": np.float32,
                "net_import": np.float32,
            }
        ),
    ]
)

In [55]:
pipeline

A pdpipe pipeline:
[ 0]  Apply dataframe method replace with kwargs {'to_replace': '.',
      'value': nan}
[ 1]  Apply dataframe method replace with kwargs {'to_replace': '...',
      'value': nan}
[ 2]  Apply dataframe method dropna with kwargs {'axis': 0, 'how': 'any'}
[ 3]  Rename columns with {'Unnamed: 0': 'year', 'Unnamed: 1': 'month',
      'Unnamed: 2': 'country', 'MWh': 'import', 'MWh.1': 'export', 'MWh.2':
      'net_import'}

In [56]:
df = pipeline.apply(df_raw).head()

In [57]:
df

,year,month,country,import,export,net_import
0,2023,Januar,Frankreich,609151,801578,-192428
1,2023,Januar,Belgien und Luxemburg,104866,738715,-633849
2,2023,Januar,Niederlande,609089,1099654,-490565
5,2023,Januar,�sterreich,497344,1407942,-910598
6,2023,Januar,Polen,33929,783748,-749819


In [58]:
.apply(df)

,year,month,country,import,export,net_import
0,2023,Januar,Frankreich,609151.0,801578.0,-192428.0
1,2023,Januar,Belgien und Luxemburg,104866.0,738715.0,-633849.0
2,2023,Januar,Niederlande,609089.0,1099654.0,-490565.0
5,2023,Januar,�sterreich,497344.0,1407942.0,-910598.0
6,2023,Januar,Polen,33929.0,783748.0,-749819.0
